RCME - 15099

In [4]:
!pip install requests
!pip install pandas
!pip install beautifulsoup4

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [5]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [6]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

In [7]:
searchTerm1 = ['BIM', 'Building Information Modelling']
searchTerm2 = ['Green Building', 'Green Buildings']


# searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
# searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
tc = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)


# https://www.tandfonline.com/action/doSearch?AllField=BIM+and+Green+Building&SeriesKey=rcme20
    
baseUrl = 'https://www.tandfonline.com/action/doSearch?AllField='
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        
        query = baseUrl + item1 +"+AND+"+ item2+"&SeriesKey=rcme20"
        # print(query)

        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        print(number)
        tc = tc + number
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
        print(pageSize)

        cnt = 0
        allLinks = []
        
        # for testing purpose only
        pageSize =1 
        for currentPage in range(0,pageSize):
            print("In page: ",currentPage)
            
            if currentPage != 0:
                newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                cnt = cnt + 1
            else:
                newQuery = query
            
            print("Searching for query: ",newQuery)
            response = requests.get(newQuery)
            soup = BeautifulSoup(response.text, 'html.parser')
            mydivs = soup.findAll("a", {'class':['ref nowrap']})
            
            for d in mydivs:
                url = 'https://www.tandfonline.com' + d.get('href')
                allLinks.append(url)

        for link in allLinks:
            df.loc[row,'searchTerm1'] = item1
            df.loc[row,'searchTerm2'] = item2
            df.loc[row,'url'] = link
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "lxml")
            
            for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
                # print("Title: ",i)
                df.loc[row,'Title'] = i
            
            temp = soup.findAll('h2',{'class':['section-heading-2']} )
            # print("Number of sections: ",len(temp))            
            
            count = 0
            for i in range(len(temp)):

                if temp[i].text == temp[i].find_next('p').text:
        
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)                    
                    df.loc[row,sectionData] = extractData
                    count = count + 1
                else:
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)
                    df.loc[row,sectionData] = extractData
                    count = count + 1
            
            
            row = row + 1
            print("row: ",row)


26
3
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rcme20
row:  1
row:  2
row:  3
row:  4
row:  5
row:  6
row:  7
row:  8
row:  9
row:  10
26
3
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=rcme20
row:  11
row:  12
row:  13
row:  14
row:  15
row:  16
row:  17
row:  18
row:  19
row:  20
315
32
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rcme20
row:  21
row:  22
row:  23
row:  24
row:  25
row:  26
row:  27
row:  28
row:  29
row:  30
315
32
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Buildings&SeriesKey=rcme20
row:  31
row:  32
row:  33
row:  34
row:  35
row:  36
row:  37
row:  38
row:  39
row:  40


In [ ]:
# df.iloc[:, 0:36]

In [ ]:
df = df.where(pd.notnull(df), None)
# df


In [ ]:
# pwd

In [ ]:
# cd Desktop/MDSOutput

In [ ]:
df.to_csv('KG-RCME-15099.csv')